In [19]:
import pandas as pd

In [20]:
# Using Excel geography functionality, I put this together
cities = pd.read_excel("../../Data/TeamLatLng/City_Lat_Lng.xlsx", usecols=["CityString", "Latitude", "Longitude"])
cities

,CityString,Latitude,Longitude
0,Atlanta,33.748547,-84.391502
1,Baltimore,39.283333,-76.616667
2,Boston,42.358056,-71.063611
3,Cincinnati,39.103697,-84.513613
4,Cleveland,41.482222,-81.669722
5,Dallas,32.775833,-96.796667
6,Denver,39.739200,-104.990300
7,Detroit,42.332941,-83.047837
8,Houston,29.762778,-95.383056
9,Kansas City,39.099722,-94.578333


In [21]:
from math import cos, asin, sqrt

# https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav))

def closest(data, v):
    return min(data, key=lambda p: distance(v['Latitude'],v['Longitude'],p['Latitude'],p['Longitude']))

# Adapted from the above - return a dict for easy mapping in
def closest_city(data:pd.DataFrame) -> dict:
    results = {}
    for city in data["CityString"].unique():
        other_cities = [x for x in data["CityString"].unique() if x != city]
        closest = min(other_cities, key=lambda other: distance(data.loc[data["CityString"]==city, "Latitude"].values[0], data.loc[data["CityString"]==city, "Longitude"].values[0],
                                                           data.loc[data["CityString"]==other, "Latitude"].values[0], data.loc[data["CityString"]==other, "Longitude"].values[0]))
        results[city] = closest
    return results

In [22]:
distance(cities.loc[cities["CityString"]=="Miami", "Latitude"].values[0], cities.loc[cities["CityString"]=="Miami", "Longitude"].values[0], cities.loc[cities["CityString"]=="Seattle", "Latitude"].values[0], cities.loc[cities["CityString"]=="Seattle", "Longitude"].values[0])

4394.608239448484

In [23]:
closest_market_map = closest_city(cities)
closest_market_map

{'Atlanta': 'Cincinnati',
 'Baltimore': 'Washington DC',
 'Boston': 'Philadelphia',
 'Cincinnati': 'Cleveland',
 'Cleveland': 'Detroit',
 'Dallas': 'Houston',
 'Denver': 'Kansas City',
 'Detroit': 'Cleveland',
 'Houston': 'Dallas',
 'Kansas City': 'St. Louis',
 'Miami': 'Tampa Bay',
 'Milwaukee': 'Detroit',
 'Minneapolis-Saint Paul': 'Milwaukee',
 'Philadelphia': 'Baltimore',
 'Phoenix': 'San Diego',
 'Pittsburg': 'Cleveland',
 'San Diego': 'Phoenix',
 'Seattle': 'Denver',
 'St. Louis': 'Kansas City',
 'Tampa Bay': 'Miami',
 'Washington DC': 'Baltimore'}

In [24]:
# Load current data
current_data = pd.read_csv("../../Data/CSVData/cleaned_current_markets_data.csv")
current_data.head()

,Market,Year,Dome,NumTeams,Attendance,Valuation,Households,Households Margin of Error,"Households Less Than $10,000","Households Less Than $10,000 Margin of Error",...,"Nonfamily Households $100,000 to $149,999 Margin of Error","Nonfamily Households $150,000 to $199,999","Nonfamily Households $150,000 to $199,999 Margin of Error","Nonfamily Households $200,000 or More","Nonfamily Households $200,000 or More Margin of Error",Nonfamily Households Median Income (Dollars),Nonfamily Households Median Income (Dollars) Margin of Error,Nonfamily Households Mean Income (Dollars),Nonfamily Households Mean Income (Dollars) Margin of Error,Nonfamily Households Nonfamily Income in the Past 12 Months
0,Atlanta,2021.0,0,1,2299647,2100000000,2187099.0,125140.0,1052.2,913.4,...,1694.4,575.6,1454.6,647.3,1472.5,8978333.0,2104775.0,12311549.0,2461058.0,6793.2
1,Atlanta,2020.0,0,1,0,1900000000,2132113.0,126396.0,1136.1,810.1,...,1545.4,505.2,1345.7,534.2,1363.9,8574443.0,1917715.0,11479625.0,2135031.0,6641.4
2,Atlanta,2019.0,0,1,2654920,1800000000,2083239.0,83709.0,1265.7,801.9,...,1440.6,436.9,1250.3,478.6,1264.4,8429231.0,1738605.0,11192645.0,1883416.0,6345.6
3,Atlanta,2018.0,0,1,2555781,1700000000,2044598.0,79769.0,1329.2,633.3,...,1437.8,417.1,1290.4,427.9,1274.8,7918807.0,1507701.0,10792810.0,1890269.0,6291.3
4,Atlanta,2017.0,0,1,2505252,1600000000,2009358.0,79349.0,1385.5,688.6,...,1399.6,371.1,1209.1,373.8,1172.8,7617423.0,1477776.0,10374040.0,1860899.0,6216.2


In [25]:
current_data["ClosestMarket"] = current_data["Market"]
current_data.loc[:, "ClosestMarket"] = current_data.loc[:, "ClosestMarket"].map(closest_market_map)

current_data.head()

,Market,Year,Dome,NumTeams,Attendance,Valuation,Households,Households Margin of Error,"Households Less Than $10,000","Households Less Than $10,000 Margin of Error",...,"Nonfamily Households $150,000 to $199,999","Nonfamily Households $150,000 to $199,999 Margin of Error","Nonfamily Households $200,000 or More","Nonfamily Households $200,000 or More Margin of Error",Nonfamily Households Median Income (Dollars),Nonfamily Households Median Income (Dollars) Margin of Error,Nonfamily Households Mean Income (Dollars),Nonfamily Households Mean Income (Dollars) Margin of Error,Nonfamily Households Nonfamily Income in the Past 12 Months,ClosestMarket
0,Atlanta,2021.0,0,1,2299647,2100000000,2187099.0,125140.0,1052.2,913.4,...,575.6,1454.6,647.3,1472.5,8978333.0,2104775.0,12311549.0,2461058.0,6793.2,Cincinnati
1,Atlanta,2020.0,0,1,0,1900000000,2132113.0,126396.0,1136.1,810.1,...,505.2,1345.7,534.2,1363.9,8574443.0,1917715.0,11479625.0,2135031.0,6641.4,Cincinnati
2,Atlanta,2019.0,0,1,2654920,1800000000,2083239.0,83709.0,1265.7,801.9,...,436.9,1250.3,478.6,1264.4,8429231.0,1738605.0,11192645.0,1883416.0,6345.6,Cincinnati
3,Atlanta,2018.0,0,1,2555781,1700000000,2044598.0,79769.0,1329.2,633.3,...,417.1,1290.4,427.9,1274.8,7918807.0,1507701.0,10792810.0,1890269.0,6291.3,Cincinnati
4,Atlanta,2017.0,0,1,2505252,1600000000,2009358.0,79349.0,1385.5,688.6,...,371.1,1209.1,373.8,1172.8,7617423.0,1477776.0,10374040.0,1860899.0,6216.2,Cincinnati


In [26]:
current_data.to_csv("../../Data/CSVData/enriched_clean_current_markets_data.csv", index=False)